# 01. 데이터 수집 - pykrx를 활용한 한국 주식 데이터 수집

## 학습 목표
- pykrx를 사용하여 다양한 종류의 주식 데이터를 수집하는 방법 학습
- 주가, 재무, 투자자별 매매 데이터 수집 및 저장
- 대량 데이터 수집 시 효율적인 처리 방법

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from pykrx import stock
import time
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# 날짜 설정
end_date = datetime.now()
start_date = end_date - timedelta(days=365*2)  # 2년 데이터

start_str = start_date.strftime('%Y%m%d')
end_str = end_date.strftime('%Y%m%d')

print(f"데이터 수집 기간: {start_str} ~ {end_str}")

데이터 수집 기간: 20230906 ~ 20250905


## 1. 종목 리스트 수집

In [2]:
def get_all_stocks():
    """
    KOSPI와 KOSDAQ 모든 종목 리스트 가져오기
    """
    stocks = []
    
    for market in ['KOSPI', 'KOSDAQ']:
        tickers = stock.get_market_ticker_list(market=market)
        print(f"{market} 종목 수: {len(tickers)}")
        
        for ticker in tqdm(tickers[:20], desc=f"{market} 종목 정보"):  # 데모용 20개
            try:
                name = stock.get_market_ticker_name(ticker)
                
                # 시가총액과 기본 정보
                cap_df = stock.get_market_cap_by_ticker(end_str, market)
                if ticker in cap_df.index:
                    cap_info = cap_df.loc[ticker]
                    
                    stocks.append({
                        'ticker': ticker,
                        'name': name,
                        'market': market,
                        'market_cap': cap_info['시가총액'],
                        'shares': cap_info['상장주식수']
                    })
                    
                time.sleep(0.1)
            except Exception as e:
                print(f"Error with {ticker}: {e}")
                continue
    
    return pd.DataFrame(stocks)

# 종목 리스트 수집
stock_list = get_all_stocks()
print(f"\n총 수집 종목: {len(stock_list)}")
print(stock_list.head(10))

KOSPI 종목 수: 961


KOSPI 종목 정보: 100%|██████████| 20/20 [00:26<00:00,  1.31s/it]


KOSDAQ 종목 수: 1805


KOSDAQ 종목 정보: 100%|██████████| 20/20 [00:37<00:00,  1.88s/it]


총 수집 종목: 40
   ticker      name market     market_cap     shares
0  095570    AJ네트웍스  KOSPI   195944446470   45252759
1  006840     AK홀딩스  KOSPI   147180402710   13247561
2  027410       BGF  KOSPI   377602740495   95716791
3  282330    BGF리테일  KOSPI  1916785175400   17283906
4  138930   BNK금융지주  KOSPI  4530866469140  314425154
5  001460       BYC  KOSPI   249846000000    6246150
6  001465      BYC우  KOSPI    56538562500    2153850
7  001040        CJ  KOSPI  5286872037600   29176998
8  079160    CJ CGV  KOSPI   793961916335  165581213
9  00104K  CJ4우(전환)  KOSPI   622565217600    4226512


## 2. 주가 데이터 수집

In [3]:
def get_price_data(ticker, start, end):
    """
    종목의 주가 데이터 수집
    
    Returns:
        DataFrame: 시가, 고가, 저가, 종가, 거래량, 등락률, 거래대금
    """
    try:
        df = stock.get_market_ohlcv_by_date(start, end, ticker)
        df['ticker'] = ticker
        
        # 추가 지표 계산
        df['MA20'] = df['종가'].rolling(window=20).mean()
        df['MA60'] = df['종가'].rolling(window=60).mean()
        df['MA120'] = df['종가'].rolling(window=120).mean()
        
        # 52주 최고/최저
        df['52주최고'] = df['고가'].rolling(window=252, min_periods=1).max()
        df['52주최저'] = df['저가'].rolling(window=252, min_periods=1).min()
        df['52주최고대비'] = df['종가'] / df['52주최고']
        
        # 거래량 관련
        df['거래량MA20'] = df['거래량'].rolling(window=20).mean()
        df['거래량비율'] = df['거래량'] / df['거래량MA20']
        
        return df
    except Exception as e:
        print(f"Error getting price data for {ticker}: {e}")
        return pd.DataFrame()

# 삼성전자 예시
samsung_price = get_price_data('005930', start_str, end_str)
print("삼성전자 주가 데이터:")
print(samsung_price.tail())
print(f"\n52주 최고가 대비: {samsung_price['52주최고대비'].iloc[-1]:.2%}")

삼성전자 주가 데이터:
               시가     고가     저가     종가       거래량       등락률  ticker     MA20  \
날짜                                                                            
2025-09-01  68400  68600  67500  67600  12002343 -3.012912  005930  70405.0   
2025-09-02  67800  69500  67800  69100  10604028  2.218935  005930  70375.0   
2025-09-03  69200  69800  68800  69800  10283009  1.013025  005930  70370.0   
2025-09-04  69500  70100  69300  70100  12284414  0.429799  005930  70435.0   
2025-09-05  70300  70400  69700  69850   3061657 -0.356633  005930  70402.5   

                    MA60         MA120    52주최고    52주최저   52주최고대비  \
날짜                                                                   
2025-09-01  65418.333333  60916.666667  80200.0  49900.0  0.842893   
2025-09-02  65573.333333  61045.000000  80100.0  49900.0  0.862672   
2025-09-03  65750.000000  61180.000000  79800.0  49900.0  0.874687   
2025-09-04  65920.000000  61306.666667  78900.0  49900.0  0.888466   
2025-09-05  6

## 3. 재무 데이터 수집

In [4]:
def get_fundamental_data(ticker, start, end):
    """
    종목의 기본적 재무 지표 수집
    
    Returns:
        DataFrame: PER, PBR, EPS, BPS, DIV, DPS
    """
    try:
        df = stock.get_market_fundamental_by_date(start, end, ticker)
        df['ticker'] = ticker
        
        # EPS 성장률 계산
        df['EPS_YoY'] = df['EPS'].pct_change(periods=252)  # 연간 성장률
        df['EPS_QoQ'] = df['EPS'].pct_change(periods=63)   # 분기 성장률
        
        # ROE 계산 (EPS / BPS)
        df['ROE'] = df['EPS'] / df['BPS'].shift(252)  # 전년도 BPS 대비
        
        return df
    except Exception as e:
        print(f"Error getting fundamental data for {ticker}: {e}")
        return pd.DataFrame()

# 삼성전자 재무 데이터
samsung_fund = get_fundamental_data('005930', start_str, end_str)
print("삼성전자 재무 지표:")
print(samsung_fund.tail())
print(f"\n최근 EPS 연간 성장률: {samsung_fund['EPS_YoY'].iloc[-1]:.2%}")
print(f"최근 ROE: {samsung_fund['ROE'].iloc[-1]:.2%}")

삼성전자 재무 지표:
              BPS    PER   PBR   EPS   DIV   DPS  ticker   EPS_YoY  EPS_QoQ  \
날짜                                                                            
2025-09-01  57951  13.66  1.17  4950  2.14  1446  005930  1.322853      0.0   
2025-09-02  57951  13.96  1.19  4950  2.09  1446  005930  1.322853      0.0   
2025-09-03  57951  14.10  1.20  4950  2.07  1446  005930  1.322853      0.0   
2025-09-04  57951  14.16  1.21  4950  2.06  1446  005930  1.322853      0.0   
2025-09-05  57951  14.10  1.20  4950  2.07  1446  005930  1.322853      0.0   

                 ROE  
날짜                    
2025-09-01  0.095189  
2025-09-02  0.095189  
2025-09-03  0.095189  
2025-09-04  0.095189  
2025-09-05  0.095189  

최근 EPS 연간 성장률: 132.29%
최근 ROE: 9.52%


## 4. 투자자별 매매 동향

In [7]:
df = stock.get_market_trading_value_by_date(start_str, end_str, '005930', detail=True)
df

,금융투자,보험,투신,사모,은행,기타금융,연기금,기타법인,개인,외국인,기타외국인,전체
날짜,,,,,,,,,,,,
2023-09-06,-65714799900,-1374596500,3741588000,-5690241900,1235827700,-18391900,8565149500,-53886800,-2481822300,62652332000,-861157900,0
2023-09-07,-27349463200,-9982734700,-4762524800,13954453500,-1210420200,4758600,-9360332200,375561900,-53537474400,92352357500,-484182000,0
2023-09-08,40827048600,-10482673000,-6350518600,-33823534300,2129121800,23526300,-1138790900,1897623000,7617478400,-714759400,15478100,0
2023-09-11,15196825900,-5153469100,1979957000,16085432300,-88861200,-27754100,3152522800,-1798713400,-81190508700,52180011200,-335442700,0
2023-09-12,27887971200,-1445961100,-8395405600,10975088000,-50130300,-60803100,486938000,-71039300,-21859798500,-7284121700,-182737600,0
...,...,...,...,...,...,...,...,...,...,...,...,...
2025-08-29,14464077350,-1494006400,-18601403850,-5243358000,104990550,-1468277400,635140250,61607897200,74272470050,-123891334900,-386194850,0
2025-09-01,-24990391950,-6000178900,-15828683500,-9479887200,1299088600,-922398500,-17607810800,62939914700,194313387650,-183584850850,-138189250,0
2025-09-02,-53818616050,2268693750,704296700,-2599832050,-363766650,1650,-1909159950,61798122100,-138118494850,132365738750,-326983400,0


In [ ]:
################################### Note #################################
# 바로 윗 셀 결과 보면 기관이 여러가지로 분할 되어 있는데 
# 이 밑에 있는 코드는 기관이라는 하나의 칼럼으로 불러오려고 하고 있어요
# 그러면 기관 순매수 칼럼의 값을 넣을때 agg를 잘 해주면 되겠죠???
# 다들 화이팅!
##########################################################################

def get_investor_trading(ticker, start, end):
    """
    투자자별 매매 동향 수집
    
    Returns:
        DataFrame: 기관, 외국인, 개인 등 투자자별 매매 데이터
    """
    try:
        # 투자자별 매매 거래량
        df = stock.get_market_trading_value_by_date(start, end, ticker, detail=True)
        df['ticker'] = ticker
        
        # 순매수 계산
        df['기관_순매수'] = df['기관'] 
        df['외국인_순매수'] = df['외국인계'] if '외국인계' in df.columns else df.get('외국인', 0)
        df['개인_순매수'] = df['개인']
        
        # 누적 순매수
        df['기관_누적'] = df['기관_순매수'].cumsum()
        df['외국인_누적'] = df['외국인_순매수'].cumsum()
        df['개인_누적'] = df['개인_순매수'].cumsum()
        
        # 최근 20일 순매수 합계
        df['기관_20일'] = df['기관_순매수'].rolling(window=20).sum()
        df['외국인_20일'] = df['외국인_순매수'].rolling(window=20).sum()
        
        return df
    except Exception as e:
        print(f"Error getting investor data for {ticker}: {e}")
        return pd.DataFrame()

# 삼성전자 투자자별 매매
samsung_investor = get_investor_trading('005930', start_str, end_str)
print("삼성전자 투자자별 매매 동향:")
print(samsung_investor.tail())
print(f"\n최근 20일 기관 순매수: {samsung_investor['기관_20일'].iloc[-1]:,.0f}")
print(f"최근 20일 외국인 순매수: {samsung_investor['외국인_20일'].iloc[-1]:,.0f}")

Error getting investor data for 005930: '기관'
삼성전자 투자자별 매매 동향:
Empty DataFrame
Columns: []
Index: []


KeyError: '기관_20일'

## 5. 업종 데이터 수집

In [8]:
def get_sector_performance():
    """
    업종별 성과 데이터 수집
    """
    # 업종 리스트
    sectors = stock.get_index_ticker_list(market='KOSPI')
    
    sector_data = []
    for sector_code in tqdm(sectors[:10], desc="업종 데이터 수집"):  # 데모용 10개
        try:
            name = stock.get_index_ticker_name(sector_code)
            
            # 업종 지수 데이터
            sector_ohlcv = stock.get_index_ohlcv_by_date(start_str, end_str, sector_code)
            
            if not sector_ohlcv.empty:
                # 성과 계산
                returns_1m = (sector_ohlcv['종가'].iloc[-1] / sector_ohlcv['종가'].iloc[-20] - 1) * 100
                returns_3m = (sector_ohlcv['종가'].iloc[-1] / sector_ohlcv['종가'].iloc[-60] - 1) * 100 if len(sector_ohlcv) > 60 else 0
                returns_6m = (sector_ohlcv['종가'].iloc[-1] / sector_ohlcv['종가'].iloc[-120] - 1) * 100 if len(sector_ohlcv) > 120 else 0
                
                sector_data.append({
                    'sector_code': sector_code,
                    'sector_name': name,
                    'returns_1m': returns_1m,
                    'returns_3m': returns_3m,
                    'returns_6m': returns_6m,
                    'last_close': sector_ohlcv['종가'].iloc[-1]
                })
                
            time.sleep(0.1)
        except Exception as e:
            print(f"Error with sector {sector_code}: {e}")
            continue
    
    return pd.DataFrame(sector_data)

# 업종 성과 데이터
sector_df = get_sector_performance()
print("\n업종별 성과:")
print(sector_df.sort_values('returns_6m', ascending=False))

업종 데이터 수집: 100%|██████████| 10/10 [00:03<00:00,  3.25it/s]


업종별 성과:
  sector_code sector_name  returns_1m  returns_3m  returns_6m  last_close
1        1002     코스피 대형주    0.240224   11.787834   26.199356     3221.39
0        1001         코스피   -0.144548   10.735433   24.899469     3205.37
4        1005      음식료·담배   -2.444252    8.666253   24.540233     4742.38
2        1003     코스피 중형주   -1.759671    7.682048   20.182425     3479.81
9        1010         비금속   -5.935414   -2.033376   12.454051     4243.15
3        1004     코스피 소형주   -2.114952   -0.468166   11.806593     2432.14
7        1008          화학   -4.200490    5.864177    8.772394     3820.13
5        1006       섬유·의류   -1.184526   -4.687627    5.663852      235.25
6        1007       종이·목재   -2.223263   -7.316323    4.983170      271.35
8        1009          제약    0.013555    2.506779   -1.885473    15642.35


## 6. 통합 데이터 수집 함수

In [9]:
class StockDataCollector:
    """
    종목 데이터를 통합적으로 수집하는 클래스
    """
    
    def __init__(self, ticker, start_date, end_date):
        self.ticker = ticker
        self.start_date = start_date
        self.end_date = end_date
        self.name = stock.get_market_ticker_name(ticker)
        
    def collect_all_data(self):
        """
        모든 종류의 데이터를 수집
        """
        print(f"\n{self.name}({self.ticker}) 데이터 수집 중...")
        
        data = {}
        
        # 1. 주가 데이터
        print("  - 주가 데이터 수집...")
        data['price'] = get_price_data(self.ticker, self.start_date, self.end_date)
        
        # 2. 재무 데이터
        print("  - 재무 데이터 수집...")
        data['fundamental'] = get_fundamental_data(self.ticker, self.start_date, self.end_date)
        
        # 3. 투자자별 매매
        print("  - 투자자별 매매 데이터 수집...")
        data['investor'] = get_investor_trading(self.ticker, self.start_date, self.end_date)
        
        return data
    
    def get_latest_metrics(self, data):
        """
        최신 지표 요약
        """
        metrics = {}
        
        if not data['price'].empty:
            metrics['현재가'] = data['price']['종가'].iloc[-1]
            metrics['52주최고대비'] = data['price']['52주최고대비'].iloc[-1]
            metrics['거래량비율'] = data['price']['거래량비율'].iloc[-1]
            metrics['20일평균대비'] = data['price']['종가'].iloc[-1] / data['price']['MA20'].iloc[-1]
        
        if not data['fundamental'].empty:
            metrics['PER'] = data['fundamental']['PER'].iloc[-1]
            metrics['PBR'] = data['fundamental']['PBR'].iloc[-1]
            metrics['EPS_YoY'] = data['fundamental']['EPS_YoY'].iloc[-1]
            metrics['ROE'] = data['fundamental']['ROE'].iloc[-1]
        
        if not data['investor'].empty:
            metrics['기관20일'] = data['investor']['기관_20일'].iloc[-1]
            metrics['외국인20일'] = data['investor']['외국인_20일'].iloc[-1]
        
        return metrics

# 사용 예시
collector = StockDataCollector('005930', start_str, end_str)
samsung_data = collector.collect_all_data()
samsung_metrics = collector.get_latest_metrics(samsung_data)

print("\n삼성전자 주요 지표:")
for key, value in samsung_metrics.items():
    if isinstance(value, float):
        if key.endswith('율') or key.endswith('YoY') or key == 'ROE' or '대비' in key:
            print(f"  {key}: {value:.2%}")
        else:
            print(f"  {key}: {value:,.2f}")
    else:
        print(f"  {key}: {value}")


삼성전자(005930) 데이터 수집 중...
  - 주가 데이터 수집...
  - 재무 데이터 수집...
  - 투자자별 매매 데이터 수집...
Error getting investor data for 005930: '기관'

삼성전자 주요 지표:
  현재가: 69800
  52주최고대비: 88.47%
  거래량비율: 27.23%
  20일평균대비: 99.15%
  PER: 14.10
  PBR: 1.2000000476837158
  EPS_YoY: 132.29%
  ROE: 9.52%


## 7. 대량 데이터 수집 및 저장

In [10]:
def collect_multiple_stocks(ticker_list, start_date, end_date):
    """
    여러 종목의 데이터를 수집하고 저장
    """
    all_data = {}
    failed_tickers = []
    
    for ticker in tqdm(ticker_list, desc="종목 데이터 수집"):
        try:
            collector = StockDataCollector(ticker, start_date, end_date)
            data = collector.collect_all_data()
            metrics = collector.get_latest_metrics(data)
            
            all_data[ticker] = {
                'name': collector.name,
                'data': data,
                'metrics': metrics
            }
            
            time.sleep(0.5)  # API 호출 제한 방지
            
        except Exception as e:
            print(f"\nError with {ticker}: {e}")
            failed_tickers.append(ticker)
            continue
    
    print(f"\n수집 완료: {len(all_data)}/{len(ticker_list)} 종목")
    if failed_tickers:
        print(f"실패한 종목: {failed_tickers}")
    
    return all_data

# 테스트: 상위 5개 종목
test_tickers = stock_list['ticker'].head(5).tolist()
collected_data = collect_multiple_stocks(test_tickers, start_str, end_str)

# 결과 요약
summary_df = pd.DataFrame([
    {
        'ticker': ticker,
        'name': data['name'],
        **data['metrics']
    }
    for ticker, data in collected_data.items()
])

print("\n수집된 데이터 요약:")
print(summary_df)

종목 데이터 수집:   0%|          | 0/5 [00:00<?, ?it/s]


AJ네트웍스(095570) 데이터 수집 중...
  - 주가 데이터 수집...
  - 재무 데이터 수집...
  - 투자자별 매매 데이터 수집...
Error getting investor data for 095570: '기관'


종목 데이터 수집:  20%|██        | 1/5 [00:05<00:20,  5.10s/it]


AK홀딩스(006840) 데이터 수집 중...
  - 주가 데이터 수집...
  - 재무 데이터 수집...
  - 투자자별 매매 데이터 수집...
Error getting investor data for 006840: '기관'


종목 데이터 수집:  40%|████      | 2/5 [00:09<00:14,  4.96s/it]

...


BGF리테일(282330) 데이터 수집 중...
  - 주가 데이터 수집...
  - 재무 데이터 수집...
  - 투자자별 매매 데이터 수집...
Error getting investor data for 282330: '기관'


종목 데이터 수집:  80%|████████  | 4/5 [00:20<00:05,  5.23s/it]


BNK금융지주(138930) 데이터 수집 중...
  - 주가 데이터 수집...
  - 재무 데이터 수집...
  - 투자자별 매매 데이터 수집...
Error getting investor data for 138930: '기관'


종목 데이터 수집: 100%|██████████| 5/5 [00:25<00:00,  5.11s/it]


수집 완료: 5/5 종목

수집된 데이터 요약:
   ticker     name     현재가   52주최고대비     거래량비율   20일평균대비   PER   PBR  \
0  095570   AJ네트웍스    4325  0.869347  0.206639  1.008452  8.92  0.44   
1  006840    AK홀딩스   11060  0.797404  0.032613  0.978674  0.00  0.27   
2  027410      BGF    3930  0.851571  0.138344  0.991423  4.08  0.22   
3  282330   BGF리테일  110300  0.849769  0.165381  0.940164  9.76  1.61   
4  138930  BNK금융지주   14480  0.902181  0.151866  0.995736  6.66  0.43   

    EPS_YoY       ROE  
0  0.321526  0.052005  
1 -1.000000  0.000000  
2  0.184502  0.055710  
3 -0.003175  0.181697  
4  0.141207  0.068481  


## 8. 데이터 저장 및 로드

In [11]:
def save_data(data, filename_prefix):
    """
    수집한 데이터를 파일로 저장
    """
    import pickle
    import os
    
    # 데이터 디렉토리 생성
    os.makedirs('data', exist_ok=True)
    
    # Pickle로 저장 (전체 데이터)
    with open(f'data/{filename_prefix}_all.pkl', 'wb') as f:
        pickle.dump(data, f)
    
    # CSV로 요약 저장
    summary_list = []
    for ticker, stock_data in data.items():
        if 'metrics' in stock_data:
            summary_list.append({
                'ticker': ticker,
                'name': stock_data['name'],
                **stock_data['metrics']
            })
    
    if summary_list:
        summary_df = pd.DataFrame(summary_list)
        summary_df.to_csv(f'data/{filename_prefix}_summary.csv', index=False, encoding='utf-8-sig')
    
    print(f"데이터 저장 완료: data/{filename_prefix}_*")

def load_data(filename_prefix):
    """
    저장된 데이터 로드
    """
    import pickle
    
    with open(f'data/{filename_prefix}_all.pkl', 'rb') as f:
        data = pickle.load(f)
    
    print(f"데이터 로드 완료: {len(data)} 종목")
    return data

# 데이터 저장
save_data(collected_data, 'canslim_demo')

# 데이터 로드 테스트
loaded_data = load_data('canslim_demo')
print(f"로드된 종목: {list(loaded_data.keys())}")

데이터 저장 완료: data/canslim_demo_*
데이터 로드 완료: 5 종목
로드된 종목: ['095570', '006840', '027410', '282330', '138930']


## 마무리

### 학습 내용 정리
1. **pykrx 주요 함수들**
   - `get_market_ticker_list()`: 종목 리스트
   - `get_market_ohlcv_by_date()`: 주가 데이터
   - `get_market_fundamental_by_date()`: 재무 지표
   - `get_market_trading_value_by_date()`: 투자자별 매매

2. **데이터 수집 시 주의사항**
   - API 호출 제한을 고려한 딜레이 추가
   - 예외 처리로 안정적인 수집
   - 대량 데이터는 저장 후 재사용

3. **계산된 지표들**
   - 이동평균선 (MA20, MA60, MA120)
   - 52주 최고/최저 대비
   - EPS 성장률 (YoY, QoQ)
   - ROE
   - 투자자별 누적 매수

### 다음 단계
- **02_current_earnings_analysis.ipynb**: C 지표 (현재 분기 실적) 분석
- 수집한 데이터를 활용한 CAN SLIM 지표 계산